## Model: Convolutional Neural Network
### Dataset - MNIST Fashion

Models included: 

| Model A | Model B   | Model C |
| ----- | ---- |
|  ReLU | ReLU | ReLU |
| 2 Convolution Layers | 2 Convolution Layers | 3 Convolution Layers |
| 2 Hidden Layers | 1 Hidden Layers | 3 Hidden Layers | 
| 91.58% | 95.76% | 96.13 |

### Step 0. Necessary Imports

In [22]:
#Necessary imports
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms 
import torchvision.datasets as dsets
from torch.autograd import Variable
import os
import gzip
import urllib.request
import os.path
import torch.utils.data as data
import codecs
from PIL import Image

### Step 1. Create and Load Dataset

#### 1.1 Dataset Class of MNIST Fashion

In [23]:
class FashionMNISTDataset(data.Dataset):
    '''Fashion MNIST Dataset'''
    
    urls = 	[
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz'
            ]

    file_name =	[
                    'train-images-idx3-ubyte',
                    'train-labels-idx1-ubyte',
                    't10k-images-idx3-ubyte',
                    't10k-labels-idx1-ubyte'
                ]
    
    training_file = 'training.pt'
    test_file = 'test.pt'
    
    def __init__(self, source , train=True, transform=None):
        
        self.source = source
        
        if os.path.exists(source) == False:
            self.download(self.source)
        
        if self.check_processed(train=train, source=source) is not True:
            self.process_files(train=train,source=source);
        
        if train is True:
            self.X, self.Y = torch.load(
                os.path.join(source, self.training_file))
        else:
            self.X, self.Y = torch.load(
                os.path.join(source, self.test_file))
        
        self.transform = transform;
        
    def __len__(self):
        return len(self.X);
    
    def check_processed(self, train, source):
        return os.path.exists(os.path.join(source, self.training_file)) and \
            os.path.exists(os.path.join(source, self.test_file))
    
    def __getitem__(self, idx):
        item = self.X[idx];
        label = self.Y[idx];
        
        item = Image.fromarray(item.numpy(), mode='L')
        
        if self.transform:
            item = self.transform(item);
        
        return (item, label);
    
    def download(self, source):
        
        try:
			      os.makedirs(self.source)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
              raise
        pass
        
        for file_index in range(len(self.file_name)):
            print("Downloading:",self.urls[file_index])
            urllib.request.urlretrieve(self.urls[file_index],(self.file_name[file_index]+'.gz'))
            f = gzip.open(self.file_name[file_index]+'.gz', 'rb')
            with open(source+self.file_name[file_index],'wb+') as w:
                for line in f.readlines():
                    w.write(line)
            f.close()
            os.remove(self.file_name[file_index]+".gz")
    
    @classmethod
    def process_files(self, train, source):
        
        if train is True:
            imgs='train-images-idx3-ubyte'
            lbls='train-labels-idx1-ubyte'
            n=60000
        else:
            imgs='t10k-images-idx3-ubyte'
            lbls='t10k-labels-idx1-ubyte'
            n=10000
        
        f = open(str(source+imgs), "rb")        
        l = open(str(source+lbls), "rb")
        
        to_set = (
            read_image_file(f),
            read_label_file(l)
        )
        
        if train is True:
            fh = open(str(source+self.training_file), 'wb+')
        else:
            fh = open(str(source+self.test_file), 'wb+')
        
        torch.save(to_set, fh)
        
        fh.close()
        
        f.close()
        l.close()

        
def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)


def read_label_file(f):
    data = f.read()
    length = get_int(data[4:8])
    parsed = np.frombuffer(data, dtype=np.uint8, offset=8)
    return torch.from_numpy(parsed).view(length).long()


def read_image_file(f):
    data = f.read()
    length = get_int(data[4:8])
    num_rows = get_int(data[8:12])
    num_cols = get_int(data[12:16])
    parsed = np.frombuffer(data, dtype=np.uint8, offset=16)
    return torch.from_numpy(parsed).view(length, num_rows, num_cols)

#### 1.2 Load Dataset

In [24]:
train_dataset = FashionMNISTDataset(source='./data/',
                                    train=True,
                                    transform=transforms.ToTensor())
test_dataset = FashionMNISTDataset(source='./data/',
                                   train=False,
                                   transform=transforms.ToTensor())

### 2. Make Dataset iterable

In [25]:
batch_size = 100
no_iters = 12000
no_epochs = no_iters / ( len(train_dataset) / batch_size )
no_epochs = int(no_epochs)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### Model A: 2 Layer CNN along with maxpool downscalling with Batch Normalization + 2 Connected Layers
#### Optimizer - SGD + Nesterov Momentum

In [20]:
#### 3. Create Model Class
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        #Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.norm1 = nn.BatchNorm2d(32)
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.norm2 = nn.BatchNorm2d(64)
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        #FNN 1
        self.fc1 = nn.Linear(64*7*7, 64*7*7)
        self.relu3 = nn.ReLU()
        
        #Linear
        self.fc2 = nn.Linear(64*7*7, 10)
        
    def forward(self,x):
        #Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.norm1(out)
        
        #Max pool 1
        out = self.maxpool1(out)
        
        #Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.norm2(out)
        
        #Max pool 2
        out = self.maxpool2(out)
        
        #Resize
        #original size: (100, 64, 7, 7)
        #out.size(0): 100
        #New out size: (100, 64*7*7)
        out = out.view(out.size(0), -1)
        
        #FNN 1
        out = self.fc1(out)
        out = self.relu3(out)
        
        #Linear
        out = self.fc2(out)
        
        return out
    
#### 4. Instantiate Model Class
if torch.cuda.is_available():
    model =CNNModel().cuda()
else:
    model = CNNModel()
    
#### 5. Instantiate Loss Class
criterion = nn.CrossEntropyLoss()

#### 6. Instantiate Optimizer Class
learning_rate = 0.01
moment = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = moment, nesterov = True)

#### 7. Train Model
iter = 1
for epoch in range(no_epochs):
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    for i,(images,labels) in enumerate(train_loader):
        #Variables
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
            
        #Clear Gradient
        optimizer.zero_grad()
        
        #Outpus
        outputs = model(images)
        
        #Calculate loss: softmax -> cross entropy loss
        loss = criterion(outputs, labels)
        
        #Generate gradients w.r.t. parameters
        loss.backward()
        
        #Update Parameters
        optimizer.step()
        
        iter += 1
        if iter %2000 ==0:
            #Calculate Accuracy
            correct=0
            total=0
            train_accuracy = 0
            test_accuracy = 0
            
            #Train Accuracy
            for images,labels in train_loader:
                if torch.cuda.is_available():
                    images = images.cuda()
                #outputs
                outputs = model(images)
                
                #Get predictions from maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
                total += labels.size(0)
                
                #Total Correct Labels
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                    
            train_accuracy = 100 * float(correct) / total
            correct=0
            total=0
            
            #Test Accuracy
            for images,labels in test_loader:
                if torch.cuda.is_available():
                    images = images.cuda()
                #outputs
                outputs = model(images)
                
                #Get predictions from maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
                total += labels.size(0)
                
                #Total Correct Labels
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                    
            test_accuracy = 100 * float(correct) / total
            
            print('Iteration: {}, Loss: {}, Train Accuracy: {}, Test Accuracy: {}'.format(iter, loss.item(), train_accuracy, test_accuracy))

Iteration: 2000, Loss: 0.12439420074224472, Train Accuracy: 94.91833333333334, Test Accuracy: 91.15
Iteration: 4000, Loss: 0.07574900984764099, Train Accuracy: 98.32333333333334, Test Accuracy: 92.11
Iteration: 6000, Loss: 0.019024420529603958, Train Accuracy: 99.54333333333334, Test Accuracy: 92.27
Iteration: 8000, Loss: 0.0004733180976472795, Train Accuracy: 99.895, Test Accuracy: 92.5
Iteration: 10000, Loss: 0.00043110846308991313, Train Accuracy: 100.0, Test Accuracy: 92.66
Iteration: 12000, Loss: 0.0005130863282829523, Train Accuracy: 100.0, Test Accuracy: 92.69


### Model B: 2 Layer CNN along with maxpool downscalling with Batch Normalization + 1 Connected Layers
#### Optimizer - SGD + Nesterov Momentum

In [15]:
#### 3. Create Model Class
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        #Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.norm1 = nn.BatchNorm2d(32)
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.norm2 = nn.BatchNorm2d(64)
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        #Linear
        self.fc = nn.Linear(64*7*7, 10)
        
    def forward(self,x):
        #Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.norm1(out)
        
        #Max pool 1
        out = self.maxpool1(out)
        
        #Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.norm2(out)
        
        #Max pool 2
        out = self.maxpool2(out)
        
        #Resize
        #original size: (100, 64, 7, 7)
        #out.size(0): 100
        #New out size: (100, 64*7*7)
        out = out.view(out.size(0), -1)
        
        #Linear
        out = self.fc(out)
        
        return out
    
#### 4. Instantiate Model Class
if torch.cuda.is_available():
    model =CNNModel().cuda()
else:
    model = CNNModel()
    
#### 5. Instantiate Loss Class
criterion = nn.CrossEntropyLoss()

#### 6. Instantiate Optimizer Class
learning_rate = 0.01
moment = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = moment, nesterov = True)

#### 7. Train Model
iter = 1
for epoch in range(no_epochs):
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    for i,(images,labels) in enumerate(train_loader):
        #Variables
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
            
        #Clear Gradient
        optimizer.zero_grad()
        
        #Outpus
        outputs = model(images)
        
        #Calculate loss: softmax -> cross entropy loss
        loss = criterion(outputs, labels)
        
        #Generate gradients w.r.t. parameters
        loss.backward()
        
        #Update Parameters
        optimizer.step()
        
        iter += 1
        if iter %2000 ==0:
            #Calculate Accuracy
            correct=0
            total=0
            train_accuracy = 0
            test_accuracy = 0
            
            #Train Accuracy
            for images,labels in train_loader:
                if torch.cuda.is_available():
                    images = images.cuda()
                #outputs
                outputs = model(images)
                
                #Get predictions from maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
                total += labels.size(0)
                
                #Total Correct Labels
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                    
            train_accuracy = 100 * float(correct) / total
            correct=0
            total=0
            
            #Test Accuracy
            for images,labels in test_loader:
                if torch.cuda.is_available():
                    images = images.cuda()
                #outputs
                outputs = model(images)
                
                #Get predictions from maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
                total += labels.size(0)
                
                #Total Correct Labels
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                    
            test_accuracy = 100 * float(correct) / total
            
            print('Iteration: {}, Loss: {}, Train Accuracy: {}, Test Accuracy: {}'.format(iter, loss.item(), train_accuracy, test_accuracy))

Iteration: 2000, Loss: 0.12232035398483276, Train Accuracy: 93.24166666666666, Test Accuracy: 90.38
Iteration: 4000, Loss: 0.09956430643796921, Train Accuracy: 95.58833333333334, Test Accuracy: 91.21
Iteration: 6000, Loss: 0.07204921543598175, Train Accuracy: 96.51333333333334, Test Accuracy: 91.01
Iteration: 8000, Loss: 0.0532103031873703, Train Accuracy: 98.50833333333334, Test Accuracy: 91.08
Iteration: 10000, Loss: 0.022581472992897034, Train Accuracy: 98.35666666666667, Test Accuracy: 91.11
Iteration: 12000, Loss: 0.059082165360450745, Train Accuracy: 99.02666666666667, Test Accuracy: 90.98


### Model C: 3 Layer CNN along with maxpool downscalling and Batch Normalization + 2 Connected Layers
#### Optimizer - SGD + Nesterov Momentum

In [26]:
#### 3. Create Model Class
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        #Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.norm1 = nn.BatchNorm2d(16)
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=1)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.norm2 = nn.BatchNorm2d(32)
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        #Convolution 3
        self.cnn3 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=3,stride=1,padding=1)
        self.relu3 = nn.ReLU()
        self.norm3 = nn.BatchNorm2d(64)
        
        #Max pool 3
        self.maxpool3=nn.MaxPool2d(kernel_size=2)

        #FNN 1
        self.fc1 = nn.Linear(64*6*6, 64*6*6)
        self.relu4 = nn.ReLU()
        
        #Linear
        self.fc2 = nn.Linear(64*6*6, 10)
        
    def forward(self,x):
        #Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.norm1(out)
        
        #Max pool 1
        out = self.maxpool1(out)
        
        #Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.norm2(out)
        
        #Max pool 2
        out = self.maxpool2(out)
        
        #Convolution 3
        out = self.cnn3(out)
        out = self.relu3(out)
        out = self.norm3(out)
        
        #Max pool 3
        out = self.maxpool3(out)
        
        #Resize
        #original size: (100, 64, 6, 6)
        #out.size(0): 100
        #New out size: (100, 64*6*6)
        out = out.view(out.size(0), -1)
        #FNN 1
        out = self.fc1(out)
        out = self.relu4(out)
        
        #Linear
        out = self.fc2(out)
        
        return out
    
#### 4. Instantiate Model Class
if torch.cuda.is_available():
    model =CNNModel().cuda()
else:
    model = CNNModel()
    
#### 5. Instantiate Loss Class
criterion = nn.CrossEntropyLoss()

#### 6. Instantiate Optimizer Class
learning_rate = 0.01
moment = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = moment, nesterov = True)

#### 7. Train Model
iter = 1
for epoch in range(no_epochs):
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    for i,(images,labels) in enumerate(train_loader):
        #Variables
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
            
        #Clear Gradient
        optimizer.zero_grad()
        
        #Outpus
        outputs = model(images)
        
        #Calculate loss: softmax -> cross entropy loss
        loss = criterion(outputs, labels)
        
        #Generate gradients w.r.t. parameters
        loss.backward()
        
        #Update Parameters
        optimizer.step()
        
        iter += 1
        if iter %2000 ==0:
            #Calculate Accuracy
            correct=0
            total=0
            train_accuracy = 0
            test_accuracy = 0
            
            #Train Accuracy
            for images,labels in train_loader:
                if torch.cuda.is_available():
                    images = images.cuda()
                #outputs
                outputs = model(images)
                
                #Get predictions from maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
                total += labels.size(0)
                
                #Total Correct Labels
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                    
            train_accuracy = 100 * float(correct) / total
            correct=0
            total=0
            
            #Test Accuracy
            for images,labels in test_loader:
                if torch.cuda.is_available():
                    images = images.cuda()
                #outputs
                outputs = model(images)
                
                #Get predictions from maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
                total += labels.size(0)
                
                #Total Correct Labels
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                    
            test_accuracy = 100 * float(correct) / total
            
            print('Iteration: {}, Loss: {}, Train Accuracy: {}, Test Accuracy: {}'.format(iter, loss.item(), train_accuracy, test_accuracy))

Iteration: 2000, Loss: 0.13402174413204193, Train Accuracy: 94.91166666666666, Test Accuracy: 91.91
Iteration: 4000, Loss: 0.12557491660118103, Train Accuracy: 97.73, Test Accuracy: 92.22
Iteration: 6000, Loss: 0.018099628388881683, Train Accuracy: 99.10666666666667, Test Accuracy: 92.31
Iteration: 8000, Loss: 0.002088089007884264, Train Accuracy: 99.97666666666667, Test Accuracy: 92.87
Iteration: 10000, Loss: 0.0002571010554675013, Train Accuracy: 100.0, Test Accuracy: 93.18
Iteration: 12000, Loss: 0.00036275864113122225, Train Accuracy: 100.0, Test Accuracy: 93.1
